# 나랑 닮은 연예인을 찾아보자
## 코드

In [1]:
import os
import numpy as np

from PIL import Image
import face_recognition

dir_path = os.getenv('HOME')+'/aiffel/data/e7_face_embedding/image'
pillow_path = os.getenv('HOME')+'/aiffel/e7_face_embedding/static/'

### Step1. 얼굴 영역 자르기

In [2]:
# 얼굴 영역 구하는 함수
def get_gropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)

    a, b, c, d = face_locations[0]
    cropped_face = image[a:c, d:b, :]  # 이미지에서 얼굴영역만 잘라냄

    return cropped_face

### Step2. 얼굴 영역의 임베딩 추출하기

In [3]:
# 얼굴 영역을 가지고 얼굴 임베딩 벡터를 구하는 함수
def get_face_embedding(face):
    return face_recognition.face_encodings(face)

# 디렉토리 안에 있는 모든 이미지의 임베딩 딕셔너리를 구하는 함수
def get_face_embedding_dict(path):
    file_list = os.listdir(path)
    embedding_dict = {}

    for file in file_list:
        file_name = file.split('.')[0]
        file_path = os.path.join(path, file)
        face = get_gropped_face(file_path)
        embedding = get_face_embedding(face)

        if len(embedding) > 0:  # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로
            embedding_dict[file_name] = embedding[0]

            pillow_image = Image.fromarray(face)
            pillow_image_path = os.path.join(pillow_path, file)
            pillow_image.save(pillow_image_path)

    return embedding_dict


# 얼굴 벡터 전환
embedding_dict = get_face_embedding_dict(dir_path)

### Step3. 모은 연예인들과 비교하기

In [6]:
# 두 얼굴 임베딩 사이의 거리 구하기
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)


def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)

    return get_distance_from_name1


def get_nearest_face(name, top=5):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x: sort_key_func(x[0]))
    result = []
    
    for i in range(top + 1):
        if i == 0: continue
        if sorted_faces[i]:
            print(f'순위 {i} : 이름({sorted_faces[i][0]}), 거리({sort_key_func(sorted_faces[i][0])})')
            result.append(sorted_faces[i][0])  # 닮은 사람 이름 반환하기 위해.
    
    return result

get_nearest_face('남은지')

순위 1 : 이름(김고은), 거리(0.38773857270573703)
순위 2 : 이름(진기주), 거리(0.38815607944658315)
순위 3 : 이름(박소담), 거리(0.3919039522329788)
순위 4 : 이름(이정현), 거리(0.40117460507622144)
순위 5 : 이름(태연), 거리(0.4123058783294155)


['김고은', '진기주', '박소담', '이정현', '태연']

### Step4. 다양한 재미있는 시각화 시도해 보기

In [7]:
"""
업로드한 파일을 가지고 닮은 연예인 찾기
"""

import os
import fnmatch
from collections import defaultdict
from werkzeug.utils import secure_filename
from flask import Flask, request, render_template


UPLOAD_DIR = os.getenv('HOME')+'/aiffel/data/e7_face_embedding/upload'
pillow_path = os.getenv('HOME')+'/aiffel/e7_face_embedding/static/'

app = Flask(__name__)
app.config['UPLOAD_DIR'] = UPLOAD_DIR

# 업로드된 이미지 작업
def check(fname):
    file_name = fname.split('.')[0]  # 파일명
    embedding = get_face_embedding_dict(UPLOAD_DIR)  # 업로드 이미지의 임베딩벡터 만들기
    img_link = defaultdict(dict)  # 업로드이미지와 연예인이미지 링크 담기
    if embedding:
        embedding_dict.update(embedding)  # 기존 임베딩 벡터에 업로드이미지 벡터 추가
        nearest_star = get_nearest_face(file_name, top=1)  # 가장 닮은 연예인 가져오기
        nearest_star = nearest_star[0]

        for file in os.listdir(pillow_path):
            if fnmatch.fnmatch(file, f'{nearest_star}.*'):
                img_link['star']['link'] = file
                img_link['star']['name'] = nearest_star
                break
    return img_link

# 파일 업로드를 위한 html 렌더링
@app.route('/')
def upload_main():
    return render_template("index.html")

# 서버에 파일 업로드
@app.route('/file-upload', methods=['POST'])
def upload_files():
    if request.method == 'POST':
        f = request.files['file']
        fname = secure_filename(f.filename)  # 사용자가 서버의 파일시스템이 있는 파일을 수정하는 것을 방지
        f.save(os.path.join(app.config['UPLOAD_DIR'], fname))
        img_link = check(fname)
        if img_link:
            return render_template("success.html", img_link=img_link)
        else:
            return render_template("fail.html")


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
 * Restarting with inotify reloader


SystemExit: 1

/home/ssac14/anaconda3/envs/aiffel/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
def show_video_in_jupyter_nb(width, height, video_url):
    from IPython.display import HTML
    return HTML("""<video width="{}" height="{}" controls>
    <source src={} type="video/mp4">
    </video>""".format(width, height, video_url))
video_url = '../e7_face_embedding/face_embedding_video.mp4'
show_video_in_jupyter_nb(600, 500,video_url)

<br><br><br><br><br>

## 회고
### 이번 프로젝트에서 **어려웠던 점**
시각화하는 부분에서 로컬에 있는 이미지를 가져오는 것이 힘들었다.  
flask를 통해 이미지를 업로드하면, 가장 유사한 연예인을 찾아주는 것을 시도하였으나, 연예인 이름은 뽑아내지만, 절대경로와 상대경로 다 테스트해봤을 때 이미지 출력을 하는데 실패했다. 그래서 좀더 찾아보니, `<img src="{{ url_for('static', filename=img_link.star.link) }}">` 이처럼 html파일에 url_for을 사용하여 진행하였더니 잘 출력이 된 것을 확인했고, static폴더 또한 server가 구동되는 파일의 같은 위치에 존재해야한다는 것을 알았다. 
  
### 만약에 루브릭 평가 관련 지표를 달성 하지 못했을 때, 이유에 관한 추정
파이참에서는 서버구동일 잘 되는데, 주피터에서는 잘 되지 않아, 파이참에서 돌린걸 영상으로 녹화해 첨부하였다.  


### 자기 다짐
현재 저 코드는 일일이 서버를 구동시킬 때마다 폴더 내 이미지들의 벡터를 일일이 매번 구하는 코드이다. 그래서 서버를 돌렸을 때, 시간이 좀 걸린다. 추후 시간이 되면 임베딩 벡터는 한번만 돌려서 저장해놓고 가져오게끔 수정해보는건 어떨까 싶다.